# Run Tutorial

In this tutorial we present how to run an MnMS simulation

## Import

In [ ]:
from mnms.demand import CSVDemandManager
from mnms.flow.MFD import Reservoir, MFDFlow
from mnms.log import attach_log_file, LOGLEVEL, get_logger, set_all_mnms_logger_level
from mnms.tools.time import Time, Dt
from mnms.graph.io import load_graph
from mnms.travel_decision.logit import LogitDecisionModel
from mnms.tools.observer import CSVUserObserver

import os
import json
import pandas as pd

## Get the parameters

In the following we read the param.json file and we retrieve all the parameters in python variables.

In [ ]:
param_file_path = "data/param.json"
param_file = open(os.getcwd() + param_file_path, 'r')
param_json = json.load(param_file)

# Get Json parameters blocs
input_params = param_json['INPUT'] # bloc with input parameters
output_params = param_json['OUTPUT'] # bloc with output parameters
supervisor_params = param_json['SUPERVISOR'] # bloc with supervisor parameters
reservoirs_params = param_json['RESERVOIRS'] # bloc with reservoirs parameters
travel_decision_params = param_json['TRAVEL_DECISION'] # bloc with travel decision parameters

param_file.close()

# Get Json parameters

# inputs
indir = input_params['indir'] # name of input folder, ex: "INPUTS"
network_file = input_params['network_file'] # path to json network file, ex: "/Lyon_symuviainput.json"
demand_file = input_params['demand_file'] # path to csv demand file 
mfd_file = input_params['mfd_file'] # path to csv MFD file
# outputs
outdir = output_params['output_dir'] # name of output folder, ex: "OUTPUTS"
log_file = output_params['log_file'] # path to log file, ex: "/simulation.log"
path_file = output_params['path_file'] # path to csv path file
user_file = output_params['user_file'] # path to csv user file
flow_file = output_params['flow_file'] # path to csv flow file
travel_time_file = output_params['travel_time_file'] # path to csv travel time file
# supervisor
log_level = supervisor_params['log_level'] # log level, ex: "LOGLEVEL.WARNING"
demand_type = supervisor_params['demand_type'] # demand type, "node" or "coordinate"
start_time = supervisor_params['start_time'] # start time in the simulation, ex: "06:30:00"
end_time = supervisor_params['end_time'] # end time in the simulation, ex: "14:00:00"
flow_dt = supervisor_params['flow_dt'] # simulation step, ex: 1 (unit in unit_flow_dt parameter)
unit_flow_dt = supervisor_params['unit_flow_dt'] # unit of the simulation step, ex: "minutes"
affectation_factor = supervisor_params['affectation_factor'] # affectation/calculation factor, ex: 10, 10 means 10 * flow_dt so 10 minutes
# reservoirs

# travel_decision
n_shortest_path = travel_decision_params['n_shortest_path'] # number of shortest path calculated
radius_sp = travel_decision_params['radius_sp'] # first radius for node search in shortest path calculation
radius_growth_sp = travel_decision_params['radius_growth_sp'] # radius step for node search in sp calculation
walk_speed = travel_decision_params['walk_speed'] # walking speed, ex: 1.4 (meter per second?)
scale_factor_sp = travel_decision_params['scale_factor_sp'] # 
algorithm = travel_decision_params['algorithm'] # algorithm used for shortest path calculation, ex: "astar" or "djikstra"
decision_model = travel_decision_params['decision_model'] # decision model used, ex: "LogitDecisionModel"
available_mobility_services = travel_decision_params['available_mobility_services'] # list with available mobility services ex: ["WALK", "PersonalCar"]

In [ ]:
set_all_mnms_logger_level(LOGLEVEL.WARNING)
#get_logger("mnms.graph.shortest_path").setLevel(LOGLEVEL.WARNING)
attach_log_file(outdir+log_file)

In [ ]:
def create_lyon_grid_multimodal():
    lyon_file_name = indir + log_file
    mmgraph = load_graph(lyon_file_name)
    return mmgraph

In [ ]:
def calculate_V_MFD(N):
    #V = 10.3*(1-N/57000) # data from fit prop
    V = 0 # data from fit dsty
    if N<18000:
        V=11.5-N*6/18000
    elif N<55000:
        V=11.5-6 - (N-18000)*4.5/(55000-18000)
    elif N<80000:
        V= 11.5-6-4.5-(N-55000)*1/(80000-55000)
    #V = 11.5*(1-N/60000)
    return max(V,0.001) # min speed to avoid gridlock

In [ ]:
def construct_reservoir_functions(data_MFD):
    func_res = dict()
    for _, row in data_MFD.iterrows():
        def res_func(dict_accumulation, pc=row["Pc"], nc=row["nc"], njam=row["njam"]):
            ncar = dict_accumulation['CAR']
            v_car = calculate_V_MFD(ncar)  # speed in m/s
            # if ncar <= nc:
            #     v_car = pc * (2 * nc - ncar) / nc ** 2
            # elif nc < ncar < njam:
            #     v_car = pc * (njam - ncar) * (njam + ncar - 2 * nc) / (njam - nc) ** 2 / ncar
            # v_car = max(v_car, 0.001)  # avoid gridlock
            v_bus = 0.15 * v_car + 10 / 3.6  # from Loder et al. 2017 (Empirics of multi-modal traffic networks)
            v_metro = 6  # dummy
            v_tram = 5  # dummy
            dict_speeds = {'CAR': v_car, 'BUS': v_bus, 'METRO': v_metro, 'TRAM': v_tram}
            return dict_speeds
        func_res[f"RES{int(row['reservoirs'])}"] = res_func
    return func_res

In [ ]:
if __name__ == '__main__':
    mmgraph = create_lyon_grid_multimodal()

    demand_file_name = indir + demand_file
    demand = CSVDemandManager(demand_file_name, demand_type=demand_type)
    demand.add_user_observer(CSVUserObserver(outdir+user_file), user_ids="all")

    flow_motor = MFDFlow(outfile=outdir+flow_file)
    res_funcs = construct_reservoir_functions(pd.read_csv(indir+mfd_file))
    for zone in mmgraph.zones:
        reservoir = Reservoir.fromZone(mmgraph, zone, res_funcs[zone])
        zone_id = zone.removeprefix('RES')
        flow_motor.add_reservoir(reservoir)

    travel_decision = LogitDecisionModel(mmgraph,
                                         outfile=outdir+path_file,
                                         n_shortest_path=n_shortest_path,
                                         radius_sp=radius_sp,
                                         radius_growth_sp=radius_growth_sp,
                                         walk_speed=walk_speed)

    supervisor = Supervisor(graph=mmgraph,
                            flow_motor=flow_motor,
                            demand=demand,
                            decision_model=travel_decision,
                            outfile=outdir+travel_time_file)

    supervisor.run(Time(start_time), Time(end_time), Dt(minutes=flow_dt), affectation_factor)